In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import lxml

req = requests.get("https://www.espncricinfo.com/records/season/team-match-results/2024-2024?trophy=89")
soup = bs(req.content)

In [2]:
summary_table = soup.find('table')
summary_df = pd.read_html(str(summary_table))[0]

<ipython-input-2-49141edd992e>:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  summary_df = pd.read_html(str(summary_table))[0]


# Getting Individual Matches

In [3]:
# Creating an array containing the links of scorecards of all matches
match_links = summary_table.find_all('a',)

href_list = []
for link in match_links:
  href = link.get('href')
  href_list.append(href)

links = href_list[1::2]

# First Match Scorecard:

In [4]:
link = "https://espncricinfo.com" + links[0]
req = requests.get(link)
soup = bs(req.content)

### First Batting Innings:

In [5]:
batting_table = soup.find_all('table', class_="ci-scorecard-table")
batting_first = batting_table[0]
batting_second = batting_table[1]

columns = batting_first.find('thead').find_all('th')
column_names = [c.string for c in columns]

table_rows = batting_first.find('tbody').find_all('tr')
l = []

for tr in table_rows:
  td = tr.find_all('td')
  row = [str(tr.get_text()).strip() for tr in td]
  l.append(row)

df = pd.DataFrame(l, columns=column_names)
df.dropna(inplace=True)
df = df.rename(columns={df.columns[1]: 'Dismissal'})

name = soup.select('h1.ds-text-title-xs.ds-font-bold.ds-mb-2.ds-m-1')
match_name = name[0].get_text()
match_name = match_name.split(',')[0]
match_name

df['Match'] = match_name

innings1 = soup.select('span.ds-text-title-xs.ds-font-bold.ds-capitalize')[0].get_text()

df['innings'] = innings1


cols = df.columns.tolist()
cols = ['Match', 'innings', 'Batting', 'Dismissal', 'R', 'B', 'M', '4s', '6s', 'SR']

df = df[cols]

### Second Batting Innings:

In [6]:
columns = batting_second.find('thead').find_all('th')
column_names = [c.string for c in columns]

table_rows = batting_second.find('tbody').find_all('tr')
l = []

for tr in table_rows:
  td = tr.find_all('td')
  row = [str(tr.get_text()).strip() for tr in td]
  l.append(row)

df2 = pd.DataFrame(l, columns=column_names)
df2.dropna(inplace=True)
df2 = df2.rename(columns={df2.columns[1]: 'Dismissal'})

name = soup.select('h1.ds-text-title-xs.ds-font-bold.ds-mb-2.ds-m-1')
match_name = name[0].get_text()
match_name = match_name.split(',')[0]
match_name

df2['Match'] = match_name

innings1 = soup.select('span.ds-text-title-xs.ds-font-bold.ds-capitalize')[1].get_text()

df2['innings'] = innings1


cols = df2.columns.tolist()
cols = ['Match', 'innings', 'Batting', 'Dismissal', 'R', 'B', 'M', '4s', '6s', 'SR']

df2 = df2[cols]

df_batting = pd.concat([df, df2], ignore_index=True)
df_batting

,Match,innings,Batting,Dismissal,R,B,M,4s,6s,SR
0,India vs South Africa,India,Rohit Sharma (c),c Klaasen b Maharaj,9,5,7,2,0,180.00
1,India vs South Africa,India,Virat Kohli,c Rabada b Jansen,76,59,87,6,2,128.81
2,India vs South Africa,India,Rishabh Pant †,c †de Kock b Maharaj,0,2,2,0,0,0.00
3,India vs South Africa,India,Suryakumar Yadav,c Klaasen b Rabada,3,4,9,0,0,75.00
4,India vs South Africa,India,Axar Patel,run out (†de Kock),47,31,38,1,4,151.61
5,India vs South Africa,India,Shivam Dube,c Miller b Nortje,27,16,32,3,1,168.75
6,India vs South Africa,India,Hardik Pandya,not out,5,2,7,1,0,250.00
7,India vs South Africa,India,Ravindra Jadeja,c Maharaj b Nortje,2,2,1,0,0,100.00
8,India vs South Africa,South Africa,Reeza Hendricks,b Bumrah,4,5,6,1,0,80.00
9,India vs South Africa,South Africa,Quinton de Kock †,c Kuldeep Yadav b Arshdeep Singh,39,31,54,4,1,125.80
